In [ ]:

import pandas as pd
import seaborn as sns
import os

In [ ]:
HC_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/HC/basic_graph_metrics.csv'
ST01_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/ST01/basic_graph_metrics.csv'
ST02_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/ST02/basic_graph_metrics.csv'
ST03_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/ST03/basic_graph_metrics.csv'

output_save_path = '/Users/jk1/temp/stroke_resilience/output/figures'

In [ ]:

HC_metrics = pd.read_csv(HC_path)
HC_metrics['timepoint_group'] = 'HC'

ST01_metrics = pd.read_csv(ST01_path)
ST01_metrics['timepoint_group'] = 'ST01'

ST02_metrics = pd.read_csv(ST02_path)
ST02_metrics['timepoint_group'] = 'ST02'

ST03_metrics = pd.read_csv(ST03_path)
ST03_metrics['timepoint_group'] = 'ST03'

all_groups_metrics = pd.concat([HC_metrics, ST01_metrics, ST02_metrics, ST03_metrics], ignore_index=True)

In [ ]:
# Drop threshold < 0.3
all_groups_metrics = all_groups_metrics[all_groups_metrics.threshold != 'top0']
all_groups_metrics = all_groups_metrics[all_groups_metrics.threshold != 'top10']
all_groups_metrics = all_groups_metrics[all_groups_metrics.threshold != 'top20']


In [ ]:
all_groups_metrics = all_groups_metrics.drop('subject', axis=1)

In [ ]:
mean_metrics = all_groups_metrics.groupby(
    ['timepoint_group', 'threshold']
).mean()
mean_metrics

In [ ]:
degree_distribution_df = mean_metrics.T
degree_distribution_df = degree_distribution_df.drop(['n_nodes', 'n_edges', 'mean_degree'])
degree_distribution_df = degree_distribution_df.astype(int)


In [ ]:
linear_df = pd.DataFrame()
for threshold in all_groups_metrics.threshold.unique():
    threshold_filter = [col for col in degree_distribution_df if threshold in col]
    HC_thresh_df = degree_distribution_df[('HC', threshold)].to_frame()
    HC_thresh_df.columns = ['degree_distribution']
    HC_thresh_df['timepoint_group'] = 'HC'

    ST01_thresh_df = degree_distribution_df[('ST01', threshold)].to_frame()
    ST01_thresh_df.columns = ['degree_distribution']
    ST01_thresh_df['timepoint_group'] = 'ST01'

    ST02_thresh_df = degree_distribution_df[('ST02', threshold)].to_frame()
    ST02_thresh_df.columns = ['degree_distribution']
    ST02_thresh_df['timepoint_group'] = 'ST02'

    ST03_thresh_df = degree_distribution_df[('ST03', threshold)].to_frame()
    ST03_thresh_df.columns = ['degree_distribution']
    ST03_thresh_df['timepoint_group'] = 'ST03'

    thresh_df = pd.concat([HC_thresh_df, ST01_thresh_df, ST02_thresh_df, ST03_thresh_df])
    thresh_df['threshold'] = threshold
    linear_df = linear_df.append(thresh_df)

In [ ]:
g = sns.displot(data=linear_df, x='degree_distribution', hue='timepoint_group', row='threshold',
            alpha=1, palette='pastel', kind="kde")

In [ ]:
g.savefig(os.path.join(output_save_path, 'degree_distribution' + ".png"))